<a href="https://colab.research.google.com/github/ramahasiba/NLP/blob/LangChain/Documents_Loading_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG


In [13]:
!pip install LangChain -q
!pip install -U langchain-community -q
!pip install pypdf -q
!pip install yt_dlp -q
!pip install pydub -q
!pip install unstructured -q
!pip install docx2txt -q
!pip install openai -q
!pip install python-dotenv -q
!pip install xlrd -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━

In [20]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [21]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

# Data Loading Libraries
# Loading PDF
from langchain.document_loaders import PyPDFLoader
# Loading csv files
from langchain.document_loaders.csv_loader import CSVLoader
# Loading Microsoft Excel Files (xls & xlsx)
from langchain.document_loaders import UnstructuredExcelLoader
# Loading Microsoft Word Files (doc / docx)
from langchain.document_loaders import Docx2txtLoader
# Loading youtube video into text
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
# Loading HTML Pages
from langchain.document_loaders import WebBaseLoader

In [4]:
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Load environment variables from a .env file
load_dotenv()

# Get the API key from the environment
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client with the API key
if openai_api_key:
    # client = OpenAI(api_key=openai_api_key)
    print("API key loaded successfully")
else:
    print("Error: OPENAI_API_KEY not found in environment variables.")

# prompt: I need llm model that is gpt 4o mini

llm_model = "gpt-4o-mini"

client = OpenAI(
    api_key=openai_api_key,
)


API key loaded successfully


In [2]:
import openai
print(openai.__version__)

1.79.0


### 1. Loading PDFs

In [5]:
pdf_file = "/content/eBook-How-to-Build-a-Career-in-AI.pdf"
loader = PyPDFLoader(pdf_file)
pages = loader.load()

Check number of pages

In [6]:
len(pages)

41

check the content of the first page

In [9]:
page = pages[0]
print(page.page_content)

PAGE 1
Founder, DeepLearning.AI
Collected Insights
from Andrew Ng
How to 
Build
Your
Career
in AI
A Simple Guide


check page metadata

In [10]:
page.metadata

{'producer': 'Adobe PDF Library 17.0',
 'creator': 'Adobe InDesign 18.0 (Macintosh)',
 'creationdate': '2022-12-13T16:08:00-05:00',
 'moddate': '2023-01-04T20:50:56+02:00',
 'trapped': '/False',
 'source': '/content/eBook-How-to-Build-a-Career-in-AI.pdf',
 'total_pages': 41,
 'page': 0,
 'page_label': '1'}

### 2. Loading CSV files

In [14]:
loader = CSVLoader(file_path = "/content/earthquake_data.csv")
data = loader.load()

Check loaded data

In [18]:
print(data[0:100])

[Document(metadata={'source': '/content/earthquake_data.csv', 'row': 0}, page_content='title: M 7.0 - 18 km SW of Malango, Solomon Islands\nmagnitude: 7\ndate_time: 22-11-2022 02:03\ncdi: 8\nmmi: 7\nalert: green\ntsunami: 1\nsig: 768\nnet: us\nnst: 117\ndmin: 0.509\ngap: 17\nmagType: mww\ndepth: 14\nlatitude: -9.7963\nlongitude: 159.596\nlocation: Malango, Solomon Islands\ncontinent: Oceania\ncountry: Solomon Islands'), Document(metadata={'source': '/content/earthquake_data.csv', 'row': 1}, page_content='title: M 6.9 - 204 km SW of Bengkulu, Indonesia\nmagnitude: 6.9\ndate_time: 18-11-2022 13:37\ncdi: 4\nmmi: 4\nalert: green\ntsunami: 0\nsig: 735\nnet: us\nnst: 99\ndmin: 2.229\ngap: 34\nmagType: mww\ndepth: 25\nlatitude: -4.9559\nlongitude: 100.738\nlocation: Bengkulu, Indonesia\ncontinent: \ncountry: '), Document(metadata={'source': '/content/earthquake_data.csv', 'row': 2}, page_content='title: M 7.0 -\nmagnitude: 7\ndate_time: 12-11-2022 07:09\ncdi: 3\nmmi: 3\nalert: green\ntsunami:

### 3. Loading Microsoft Excel Files (xls & Xlsx)

In [25]:
loader = UnstructuredExcelLoader("/content/star_schema_data2.xlsx", mode="elements")
docs = loader.load()

In [64]:
docs[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.0 (Macintosh)', 'creationdate': '2022-12-13T16:08:00-05:00', 'moddate': '2023-01-04T20:50:56+02:00', 'trapped': '/False', 'source': '/content/eBook-How-to-Build-a-Career-in-AI.pdf', 'total_pages': 41, 'page': 0, 'page_label': '1'}, page_content='PAGE 1\nFounder, DeepLearning.AI\nCollected Insights\nfrom Andrew Ng\nHow to \nBuild\nYour\nCareer\nin AI\nA Simple Guide')

### 4. Loading Microsoft Word Files (doc / docx)

In [28]:
from langchain.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("/content/SpatialFinal (1).docx")
data = loader.load()

In [29]:
data[0].page_content

'Geometries\xa0\n\nAfter understanding the coordinates systems, we can define how geometries can be described using these coordinate systems. Our goal now is to explain:\xa0\n\n\xa0simple features, a standard that describes point, line, and polygon geometries along with operations on them,\xa0\n\noperations on geometries,\xa0\xa0\xa0\n\ncoverages, functions of space or space-time,\xa0\n\ntesselations, sub-divisions of larger regions into sub-regions, and\xa0\n\nnetworks.\xa0\n\n\n\n1. Simple feature geometries:\n\n\n\nThey are a way to describe the geometries of features. features are defined previously as things that have a geometry, potentially implicitly some time properties, and further attributes that could include labels describing the thing and/or values quantitatively measuring it.\xa0\n\n\n\nThe main application of simple feature geometries is to describe geometries in two-dimensional space by points, lines, or polygons. The “simple” adjective refers to the fact that the line 

### 5. Loading Youtube Videos into Text

load the video, then convert the audio into text

In [31]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

url = "https://www.youtube.com/watch?v=TSjGRbk3l28&t=32s"
save_dir = "/content/data"
loader = GenericLoader(
    YoutubeAudioLoader([url], save_dir),
    OpenAIWhisperParser(api_key=openai_api_key)
)

docs = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=TSjGRbk3l28&t=32s
[youtube] TSjGRbk3l28: Downloading webpage
[youtube] TSjGRbk3l28: Downloading tv client config
[youtube] TSjGRbk3l28: Downloading player b2858d36-main
[youtube] TSjGRbk3l28: Downloading tv player API JSON
[youtube] TSjGRbk3l28: Downloading ios player API JSON
[youtube] TSjGRbk3l28: Downloading m3u8 information
[info] TSjGRbk3l28: Downloading 1 format(s): 140
[download] Destination: /content/data/LangChain in 2 minutes.m4a
[download] 100% of    2.41MiB in 00:00:00 at 3.24MiB/s   
[FixupM4a] Correcting container of "/content/data/LangChain in 2 minutes.m4a"
[ExtractAudio] Not converting audio /content/data/LangChain in 2 minutes.m4a; file is already in target format m4a
Transcribing part 1!


In [63]:
# !pip install python-docx

In [40]:
from docx import Document
from google.colab import files

# Assuming 'data' from the previous cell contains the loaded Word document content
if 'data' in globals() and docs:
    doc = Document()
    # Assuming data is a list of Document objects, take the content from the first one
    # Adjust this logic if 'data' has a different structure or if you want to combine contents
    for page in docs:
        doc.add_paragraph(page.page_content)

    word_filename = "output_document.docx"
    doc.save(word_filename)

    # Download the created Word file|
    files.download(word_filename)
else:
    print("No document data found in the variable 'data'.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 6. Loading HTML Pages

In [51]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://github.com/ramahasiba/Clustering_Analysis")
docs = loader.load()

In [53]:
print(docs[0].page_content[6500:7000])

nder of the participating family members.
As indicated in Table (1), the data comprises two categories of missing data: natural missing data (NaN) and missing data due to particular reasons, with each category having a label that we processed differently. We maintained all columns except those that were eliminated following discussion:
Columns with more than 75% missing data
Columns with 65% missing data for a specified cause.




Label
Description




Not required under the circumstances
995





### storing different file extensions that the application can expect to handle and the corresponding data handler, this is for being able to work with all different file types that LangChain can use.


In [60]:
DOCUMENT_MAP = {
    ".csv": CSVLoader,
    ".xls": UnstructuredExcelLoader,
    ".xlsx": UnstructuredExcelLoader,
    ".docx": Docx2txtLoader,
    ".doc": Docx2txtLoader,
    ".pdf": PyPDFLoader,
    }

### This code is used to detect the extension of the file and based on this, I can use the right data loader from langchain:

In [62]:
docx_file_path = "/content/eBook-How-to-Build-a-Career-in-AI.pdf"
file_extension = os.path.splitext(docx_file_path)[1]
loader_class = DOCUMENT_MAP.get(file_extension)

loader = loader_class(docx_file_path)
docs = loader.load()